### Model Compressor Example
This example demonstrates how to extend from the class `Compressor` to create a new 
compressor model. 


First, let's import the compressor base model from vclibpy's
`compressor` package as well as the `Inputs` class:


In [14]:
from vclibpy.components.compressors import Compressor
from vclibpy.datamodels import Inputs


Our new class `ConstantVolumeRatioCompressor` inherits from the base class `Compressor`. 


In [15]:
class ConstantVolumeRatioCompressor(Compressor): 
    #We need an initialization method of the new model. When we initialize the class ‘ConstantVolumeRatioCompressor’, 
    #we have to initialize the inherited parameters N_max and V_h as well as the model specific parameters v_r, eta_mech and lambda_h.
    
    def __init__(self,
           N_max: float, V_h: float, v_r: float, lambda_h: float, eta_mech: float):
        super().__init__(N_max=N_max, 
                         V_h=V_h)
        self.v_r = v_r
        self.lambda_h = lambda_h
        self.eta_mech = eta_mech
    #Now, we have to implement the methods `get_lambda_h`,`get_eta_mech` and `eta_isentropic`. 
    #`get_lambda_h`,`get_eta_mech` return the specified patameters `eta_mech` and `lambda_h`.    
    def get_eta_mech(self, inputs: Inputs) -> float:

        return self.eta_mech


    def get_lambda_h(self, inputs: Inputs) -> float:

        return self.lambda_h
    
    #Now we implement the method eta_isentropic. We need the density of the fluid at the inlet of the compressor to calculate the density at the outlet.
    
    def get_eta_isentropic(self, p_outlet: float, inputs: Inputs):
        
        d1 = self.state_inlet.d
        d2 = d1/ self.v_r    
        
        #Then we calculate the enthalpy of the outlet state of the compressor.
        
        h2 = self.med_prop.calc_state("PD", p_outlet, d2).h
        h2_is = self.med_prop.calc_state("PS", p_outlet, self.state_inlet.s).h
        h1 = self.state_inlet.h
        
        # The isentropic efficiency is now calculated and returned:
        eta_isentropic = (h2-h1)/(h2_is-h1)
        
        return eta_isentropic
    

Now we can use the model as shown in the following: 



### Application Example


Let's specify some dummy parameters:


In [27]:
compressor = ConstantVolumeRatioCompressor(N_max=120,
                                           V_h=19e-6,
                                           v_r=1.1, 
                                           lambda_h= 0.9,
                                           eta_mech=0.9)


Before we can do any calculations, the compressor needs
access to refrigerant data. Each component in VcLibPy has
the property med_prop (media property) which you can set like this:

In [28]:
from vclibpy.media import CoolProp
med_prop = CoolProp(fluid_name="Propane",use_high_level_api = True)
compressor.med_prop = med_prop

Now, you have to define the input state of the compressor.
Each component has inlet and outlet states, which are, same as `med_prop`
properties of the component.
We assume a super-heated vapor at 1bar as an input state:


In [29]:
p_inlet = 1e5
T_superheat = med_prop.calc_state("PQ", p_inlet, 1).T + 10
compressor.state_inlet = med_prop.calc_state("PT", p_inlet, T_superheat)


We set the compressor speed to 50 % of the maximum speed. 

In [30]:
from vclibpy import FlowsheetState
fs_state = FlowsheetState()
inputs = Inputs(n=0.5)


We define the pressure at the compressor's outlet and calculate the outlet state: 

In [31]:
p_outlet = 6e5
compressor.calc_state_outlet(p_outlet=p_outlet, inputs=inputs, fs_state=fs_state)
print(f"{compressor.state_outlet=}")  # now calculated
print(f"{compressor.state_outlet.T=}")

compressor.state_outlet=<vclibpy.media.states.ThermodynamicState object at 0x000001FD1FE1E1F0>
compressor.state_outlet.T=281.07408524698366


We can also calculate the mass flow. 

In [32]:
m_flow = compressor.calc_m_flow(inputs=inputs, fs_state=fs_state)
print(f"{m_flow=}, {compressor.m_flow=}")

m_flow=0.0023351945713373317, compressor.m_flow=0.0023351945713373317


Also, relevant results are automatically added to the fs_state:

In [33]:
print(fs_state)

FlowsheetState:
eta_is=52.27344677156065 % (Isentropic efficiency)
lambda_h=0.9 % (Volumetric efficiency)
V_flow_ref=0.0010260000000000002 m3/s (Refrigerant volume flow rate)
m_flow_ref=0.0023351945713373317 kg/s (Refrigerant mass flow rate)
